In [31]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

import nltk
from nltk.corpus import stopwords
from b2w_reviews import (text_process, tests)


from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

datetime.now().strftime("%d/%m/%Y %H:%M:%S")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'28/02/2023 03:56:17'

In [2]:
# nltk.download()

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', None)

pd.options.plotting.backend = "plotly"

# Load

In [4]:
df = pd.read_csv(r"data\B2W-Reviews01.csv", low_memory=False)

df[:5]

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


# Remove duplicates - test_nunique

In [5]:
df.shape

(132373, 14)

In [6]:
df = (
    df
    .drop_duplicates(subset=["reviewer_id"], keep="first")
)

df.shape

(112993, 14)

In [7]:
tests.test_reviewer_id_nunique(df)

2023-02-28 03:24:21,672 - b2w_reviews.logs - INFO - Running test_reviewer_id_nunique
2023-02-28 03:24:21,731 - b2w_reviews.logs - INFO - Finished test_reviewer_id_nunique


------

# Text processing

In [8]:
df = (
    df
    .fillna({"review_text": "-"})
    .assign(review_text_preprocessed=lambda df: df["review_text"].apply(text_process.clean_text))
    .assign(review_text_preprocessed=lambda df: text_process.remove_punctuation(df["review_text_preprocessed"]))
    .assign(review_text_preprocessed=lambda df: text_process.remove_stopwords(df["review_text_preprocessed"]))
    .assign(review_text_preprocessed=lambda df: text_process.stemming(df["review_text_preprocessed"]))
    .assign(review_text_preprocessed_tokens=lambda df: text_process.tokenize(df["review_text_preprocessed"]))    
)

df[:5]

2023-02-28 03:24:24,730 - b2w_reviews.logs - INFO - Running remove_punctuation
2023-02-28 03:24:25,915 - b2w_reviews.logs - INFO - Finished remove_punctuation
2023-02-28 03:24:26,020 - b2w_reviews.logs - INFO - Running remove_stopwords
2023-02-28 03:24:29,145 - b2w_reviews.logs - INFO - Finished remove_stopwords
2023-02-28 03:24:29,318 - b2w_reviews.logs - INFO - Running stemming
2023-02-28 03:24:55,208 - b2w_reviews.logs - INFO - Finished stemming
2023-02-28 03:24:55,329 - b2w_reviews.logs - INFO - Running tokenize
2023-02-28 03:25:05,934 - b2w_reviews.logs - INFO - Finished tokenize


,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,review_text_preprocessed,review_text_preprocessed_tokens
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ,content compra entrega rápida único problema a...,"[content, compra, entrega, rápida, único, prob..."
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC,apena r199420eu consegui comprar lindo copo ac...,"[apena, r199420eu, consegui, comprar, lindo, c..."
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP,supera agilidad praticidad outra panela elétri...,"[supera, agilidad, praticidad, outra, panela, ..."
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP,filho amou parec verdad tanto detalh têm,"[filho, amou, parec, verdad, tanto, detalh, têm]"
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG,entrega prazo americana parabén smart tv boa n...,"[entrega, prazo, americana, parabén, smart, tv..."


-------

In [9]:
# período da base
df["submission_date"].agg(["min", "max"])

min    2018-01-01 00:11:28
max    2018-05-31 23:50:33
Name: submission_date, dtype: object

## Check reviews

In [37]:
# distribuição de recommend_to_a_friend
(
    df
    ["recommend_to_a_friend"]
    .value_counts(normalize=True)
)

Yes    0.72477
No     0.27523
Name: recommend_to_a_friend, dtype: float64

In [10]:
# distribuição de notas no review
(
    df
    ["overall_rating"]
    .value_counts()
    .sort_index()
    # .plot(kind="bar")
)

1    23648
2     7334
3    13836
4    28073
5    40102
Name: overall_rating, dtype: int64

In [11]:
# distribuição de notas no review

def rename_col_groupby(df):
    """ 
    Rename columns appending _func to column name.
    Source: https://stackoverflow.com/questions/19078325/naming-returned-columns-in-pandas-aggregate-function/43897124
    """
    df.columns = ["_".join(c) for c in df.columns.ravel()]
    return df


plot_reviews_over_time = (
    df
    .assign(submission_date=lambda df: pd.to_datetime(df["submission_date"]))
    .astype({"overall_rating": str})
    .groupby([pd.Grouper(key="submission_date", freq="MS"), "overall_rating"])
    .agg({
        "reviewer_id": "count"
    })
    .unstack(["overall_rating"])
    .pipe(lambda df: rename_col_groupby(df))
)

plot_reviews_over_time

C:\Users\jpfmi\AppData\Local\Temp\ipykernel_18656\1020199160.py:8: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  df.columns = ["_".join(c) for c in df.columns.ravel()]


,reviewer_id_1,reviewer_id_2,reviewer_id_3,reviewer_id_4,reviewer_id_5
submission_date,,,,,
2018-01-01,5778,1879,3838,8138,11853
2018-02-01,2211,812,1234,2424,3539
2018-03-01,5506,1628,3082,6234,9041
2018-04-01,5336,1514,3105,6226,8708
2018-05-01,4817,1501,2577,5051,6961


In [12]:
fig = make_subplots(
    x_title="Data",
    y_title="Avaliações"
)

for col in plot_reviews_over_time.columns:

    fig.add_trace(
        go.Scatter(
            x=plot_reviews_over_time.index,
            y=plot_reviews_over_time[col].values,
            mode='markers+lines',
            name=f"Avaliações {col[-1:]}",
        ),
        row=1, col=1
    )

graph_paramns = {
    'hovermode': 'x unified',
    'legend': {'orientation': 'h', 'y': 1.18},
    "yaxis2": {"title": ""},
    "title": f"""
    Avaliações - Tempo
    """,
}
fig.update_layout(**graph_paramns)
fig.show()

In [13]:
# comparar notas com feature de recomendacao para amigos
(
    df
    .groupby(["overall_rating", "recommend_to_a_friend"])
    .agg({
        "reviewer_id": "count"
    })
    .assign(percent=lambda df: df.groupby(level=[0])["reviewer_id"].transform(lambda x: x/x.sum()))
)

reviewer_id   percent
overall_rating recommend_to_a_friend                       
1              No                           22973  0.971456
               Yes                            675  0.028544
2              No                            5558  0.757840
               Yes                           1776  0.242160
3              No                            1602  0.115793
               Yes                          12233  0.884207
4              No                             403  0.014355
               Yes                          27670  0.985645
5              No                             562  0.014015
               Yes                          39537  0.985985

---

## Check produtos

In [16]:
df["product_id"].nunique()

41498

In [17]:
# check 1:1 relation
df.groupby(["product_id"])["product_name"].nunique().sort_values(ascending=False)[:3]

product_id
10003394    1
26776701    1
26770648    1
Name: product_name, dtype: int64

In [18]:
# top products
(
    df
    .groupby(["product_id", "product_name"])
    .size()
    .sort_values(ascending=False)
    [:10]
)

product_id  product_name                                                                                                                                
132444092   Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum                            730
126474851   Aspirador de Pó Philco Rapid 1000N Vermelho/Preto - 1000W                                                                                       657
128010777   Smartphone Samsung Galaxy J7 Metal Dual Chip Android 6.0 Tela 5.5" 16GB 4G Câmera 13MP - Dourado                                                613
132444050   Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Dourado                             517
129543938   Smartphone Samsung Galaxy J7 Prime Dual Chip Android Tela 5.5" 32GB 4G Câmera 13MP - Dourado                                                    476
132276480   Smartphone Samsung Galaxy J5 Prime 

## Check brands

In [19]:
df["product_brand"].isnull().mean()

0.6834405671147771

In [20]:
df["product_brand"].value_counts()[:5]

samsung    5458
lg         2645
philco     2397
cadence    1198
philips    1118
Name: product_brand, dtype: int64

## Check categories

In [22]:
product_categories = (
    df
    .groupby(["site_category_lv1"])
    .agg({
        "product_id": "nunique"
    })
    .sort_values(by=["product_id"], ascending=False)
)

product_categories[:5]

,product_id
site_category_lv1,
Móveis,3398
Beleza e Perfumaria,2965
Utilidades Domésticas,2634
Informática e Acessórios,2524
Livros,2419


In [23]:
(
    df
    .pipe(lambda df: df[df["site_category_lv1"].isin(product_categories.index[:5])])
    .groupby(["site_category_lv1", "site_category_lv2"])
    .agg({
        "product_id": "nunique"
    })
    .sort_values(by=["site_category_lv1", "product_id"], ascending=[True, False])
    [:5]
)

product_id
site_category_lv1   site_category_lv2               
Beleza e Perfumaria Cabelo                       818
                    Perfumaria                   507
                    Produtos para cabelo         409
                    Barbearia                    231
                    Maquiagem                    201

In [24]:
def plot_subcategories(df, categorie):
    fig = make_subplots(
        y_title="Produtos únicos",
        x_title="Subcategorias de produtos"
    )

    df = (
        df
        # .pipe(lambda df: df[df["site_category_lv1"].isin(product_categories.index[:5])])
        .groupby(["site_category_lv1", "site_category_lv2"])
        .agg({
            "product_id": "nunique"
        })
        .sort_values(by=["site_category_lv1", "product_id"], ascending=[True, False])
        .loc[f"{categorie}"]
        .reset_index()
    )

    fig.add_trace(
        go.Bar(
            x=df["site_category_lv2"],
            y=df["product_id"],
        ),
        row=1, col=1
    )

    graph_paramns = {
        'hovermode': 'x unified',
        'legend': {'orientation': 'h', 'y': 1.18},
        "title": f"""
        Subcategories - {categorie}
        """,
    }
    fig.update_layout(**graph_paramns)
    return fig.show()

In [25]:
plot_subcategories(df=df, categorie="Beleza e Perfumaria")

-------

## Check review_text

In [26]:
# tamanho das reviews
(
    df
    .assign(review_text_len=lambda df: df["review_text"].astype(str).apply(len))
    [["review_text", "review_text_len"]]
    .sort_values(by=["review_text_len"], ascending=False)
    [:5]
)

,review_text,review_text_len
106209,ele e whrgeyretr36ryt3r3bh3irj3uy4rur3ir5irhgr...,4548
41017,pena que não tem desconto pena que não tem de...,4134
131513,Porque não recomendo este produto: 1) O produ...,3119
85105,"Já tive vários aparelhos da Motorola, e como s...",2766
117676,O produto é realmente muito bom. Como não tenh...,2685


In [27]:
describe_review_text_len = (
    df
    .assign(review_text_len=lambda df: df["review_text"].astype(str).apply(len))
    ["review_text_len"]
    .describe(percentiles=[0.50, 0.75, 0.90, 0.95, 0.99])
)

describe_review_text_len    

count    112993.000000
mean        133.537963
std         125.033351
min           1.000000
50%          94.000000
75%         153.000000
90%         258.000000
95%         351.000000
99%         638.080000
max        4548.000000
Name: review_text_len, dtype: float64

In [28]:
plot_hist_reviews_len = (
    df
    .assign(review_text_len=lambda df: df["review_text"].astype(str).apply(len))
    .pipe(lambda df: df[df["review_text_len"] <= describe_review_text_len.loc["99%"]])
    # ["review_text_len"]
    # .plot(kind="hist")
)

plot_hist_reviews_len[["review_text", "recommend_to_a_friend", "review_text_len"]][:5]

,review_text,recommend_to_a_friend,review_text_len
0,Estou contente com a compra entrega rápida o ú...,Yes,158
1,"Por apenas R$1994.20,eu consegui comprar esse ...",Yes,69
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,Yes,236
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,Yes,62
4,"A entrega foi no prazo, as americanas estão de...",Yes,202


In [29]:
fig = make_subplots(
    y_title="Ocorrências",
    x_title="Tamanho da review"
)

fig.add_trace(
    go.Histogram(
        x=(
            plot_hist_reviews_len
            [["review_text", "recommend_to_a_friend", "review_text_len"]]
            .pipe(lambda df: df[df["recommend_to_a_friend"] == "No"])
            .pipe(lambda df: df[df["review_text_len"] <= describe_review_text_len.loc["99%"]])
            ["review_text_len"]
            .values
        ),
        name="Bad reviews",
    ),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(
        x=(
        plot_hist_reviews_len
        [["review_text", "recommend_to_a_friend", "review_text_len"]]
        .pipe(lambda df: df[df["recommend_to_a_friend"] == "Yes"])
        .pipe(lambda df: df[df["review_text_len"] <= describe_review_text_len.loc["99%"]])
        ["review_text_len"]
        .values
    ),
        name="Good reviews",
    ),
    row=1, col=1
)

graph_paramns = {
    'hovermode': 'x unified',
    'legend': {'orientation': 'h', 'y': 1.18},
    "yaxis2": {"title": ""},
    "title": f"""
    Histograma Reviews
    """,
}
fig.update_layout(**graph_paramns)
fig.show()

In [32]:
most_common_words_counter = Counter([item for sublist in df['review_text_preprocessed_tokens'] for item in sublist])
most_common_words_counter_neg = Counter([item for sublist in df.pipe(lambda df: df[df["recommend_to_a_friend"] == "No"])['review_text_preprocessed_tokens'] for item in sublist])
most_common_words_counter_pos = Counter([item for sublist in df.pipe(lambda df: df[df["recommend_to_a_friend"] == "Yes"])['review_text_preprocessed_tokens'] for item in sublist])

most_common_words = pd.DataFrame(most_common_words_counter.most_common(20), columns=["word", "freq"])
most_common_words_neg = pd.DataFrame(most_common_words_counter_neg.most_common(20), columns=["word", "freq"])
most_common_words_pos = pd.DataFrame(most_common_words_counter_pos.most_common(20), columns=["word", "freq"])

most_common_words[:5]

,word,freq
0,produto,61927
1,bom,21306
2,recomendo,19983
3,entrega,19489
4,bem,14067


In [34]:
most_common_words_neg[:5]

,word,freq
0,produto,20321
1,dia,5991
2,recebi,5916
3,comprei,5444
4,americana,4229


In [35]:
most_common_words_pos[:5]

,word,freq
0,produto,41602
1,bom,19815
2,recomendo,17337
3,entrega,15761
4,bem,12310


In [ ]:
## produto pode ser uma palavra problematica, possivelmente ele só faz sentido se estiver dentro de n-grams. Podemos avaliar de remover essa palavra e outras para serem adicionadas como stopwords.

In [41]:
fig = px.treemap(most_common_words, path=['word'], values='freq', title='Freq Palavras')
fig.show()